In [1]:
from IPython.display import Image

# Kafka Tutorial 1
[카프카 공식 링크](http://kafka.apache.org/)

0. [용어정리](#0.-용어정리)
1. [Kafka](#1.-Kafka-%3F)
2. [Kafka 구조](2.-Kafka-구조)
3. [Kafka 특징](#3.-카프카-특징)
4. [Kafka 구성요소 (Core concepts)](#4.-구성요소-(Core-Concept))
5. [Example: Hello Kafka!](#5.-Hellow-Kafka!)
6. [요약](#6.-요약)
7. [부록, 카프카 관리 스크립트](#Appendix.-카프카-관리-스크립트)

## 0. 용어정리
- Pub-Sub 모델: pub-sub 모델에서는 메시지를 특정 수신자에게 직접적으로 보내주는 시스템이 아니다. 'topic' 이란 메시지를 카테고리화한 후, 그곳에 publisher가 메시지를 쓰고 subscriber가 그 'topic' 을 바라보면서 그것 안의 메시지를 읽는것이다.
- Message, record: 주고받는 데이터의 단위
- Producer, Publisher: 레코드를 생산(발행)하는 주체
- Consumer, Subscriber: 레코드를 소비(구독)하는 주체
- Topic: 메시지(레코드)가 종류별로 쌓이는 공간
- Partition: 분산처리를 위해 토픽을 논리적으로 분할해놓은 공간
- batch: 특정 토픽에 쌓일 레코드의 묶음 

## 1. Kafka
![Apache Kafka](./assets/Apache_Kafka.png)  
소개
- 아파치 카프카(Kafka)는 Pub-Sub 모델을 기반으로하여 스칼라로 개발된 오픈소스 메시지 큐 이다.
- 분산환경에 특화되어 설계되었기에 기존의 메시지 큐(ActiveMQ, RabbitMQ)보다 월등한 처리량을 보인다. 

사용사례
- 주로 큰 메시지를 (신뢰할 수 있게, 내장애성(일부 장애 발생시에도 정상적으로 운영) 방식으로 메시지 스트림을 저장함) 처리하는 실시간 스트리밍 데이터 파이프라인을 만들때 카프카를 사용한다. 
- 분산된 아키텍쳐의 구조적인 복잡도를 줄이고자 사용한다.  

신뢰성
- 2011년 첫 릴리즈이후 지속적으로 빠르게 성장.
- 포츈 1/3 이상 기업들이 카프카를 사용.
---

## 2. Kafka 구조
![kafka_api](./assets/kafka-apis.png)

이 구조는 카프카 클러스터(카프카 브로커)와, 컨슈머, 프로듀서가 결합된 간단한 구조로 볼 수 있다.

![kafka_structure](./assets/kafka_structure_1.png)  


- 카프카 브로커는 대용량 실시간 스트리밍 데이터를 중단 없이 처리하기 위해 **고가용성(High Availability)**과 **스케일아웃**이 고려되서 설계되었다.  

> 고가용성: 클러스터가 오랜시간동안 안정적으로 운영되는 성질. 클러스터 내 한 서버가 다운되는 등 장애가 발생하더라도 다른 서버가 그 서비스를 대신 수행함. 

> 스케일아웃: 클러스터내 서버를 수평적으로 확장시키는 것.

- 따라서 아래와 같이 카프카 클러스터(분산된 개별 카프카 브로커들의 묶음)를 안정적으로 운영하기 위해 ***주키퍼 클러스터***가 카프카 클러스터에 붙여져서 카프카를 관리하게 된다.

> 주키퍼: 분산 시스템의(카프카 클러스터의) 고가용성과 스케일 아웃을 지원하기 위해 만들어진 네이밍 레지스트리 시스템

![kafka-cluster-step2](./assets/kafka_cluster.png)

---

## 3. 카프카 특징
- 다중 프로듀서

> 다중 프로듀서는 한개 이상의 토픽에 메시지들을 발행할 수 있으며 컨슈머의 존재를 몰라도 된다.

- 다중 컨슈머

> 컨슈머들은 한개 이상의 토픽의 메시지들을 구독할 수 있으며, 한 컨슈머가 특정 메시지를 읽은 경우 다른 컨슈머들이 해당 메시지를 읽지 못하게(*중복되지않게*) 카프카 브로커 내부에서 조율한다. 즉, 하나의 메시지는 한번만 처리게되 유지된다.

> 브로커가 컨슈머에게 push 하는 방식이 아닌, 컨슈머가 브로커에게 pull하는 방식이라 컨슈머의 처리 능력에 따라 효율적으로 처리될 수 있다. 

- 무중단(장비의 장애)

> 카프카 클러스터는 주키퍼에 의해 관리되어진다. 한 브로커에 장애가 발생하면 그것을 감지하여 다른 브로커가 대신 Pub-Sub 서비스를 계속해서 수행한다.

- 디스크 기반 보존(Disk-Based Retention)

> 카프카의 주된 장점중 하나로 컨슈머들이 실시간으로 토픽에서 메시지들을 가져오지 않아도 **카프카 브로커가 메시지들을 디스크에 저장(commit)**한다. 컨슈머가 예상치 못한 오류로 인해 실패해도 메시지를 다시 읽어갈 수 있으며 이로 인해 메시지 유실 가능성이 급격하게 줄어든다.

- 간단한 스케일 아웃

> 필요에 따라(데이터 처리량 증가 등) 클러스터가 온라인인 상태에서도 브로커를 유연하게 추가하거나 제거할 수 있다. 

> 이는 개별적인 카프카 브로커들의 장애를 견딜 수 있음을 의미한다(replication 구성을 할 경우). 

- 고성능 (High Throghput)

> 위의 장점들 덕분에 카프카는 요구사항에 따라 브로커, 프로듀서, 컨슈머 수를 자유롭게 늘림으로서 매우 큰 메시지 스트릠을 실시간으로 빠르게 처리(High Throghput)할 수 있다.

---

## 4. 구성요소 (Core Concept)
카프카를 크게보면 주키퍼와 카프카 브로커, 토픽, 프로듀서, 컨슈머 이 4가지 부분으로 구성된다.

1. 쥬키퍼, 카프카 브로커
2. 프로듀서
3. 토픽과 파티션, 오프셋
4. 컨슈머, 컨슈머 그룹


### 4.1. 쥬키퍼와 카프카 브로커
![kafka-zookeeper](./assets/kafka_zookeeper.png)

#### 주키퍼 개요
주키퍼는 카프카 클러스터(분산 처리 시스템)를 안정적으로 운영되게끔 도와주는 네이밍 레지스트리 시스템이고 다음과 같은 역활을 한다.
- **설정 관리**: 카프카 브로커 서버들에 관한 메타데이터(환경 설정 등)를 최신으로 유지 및 통합 관리
    - 컨슈머 메타데이터와 파티션 오프셋 관리
    - 카프카 브로커와 토픽의 메타데이터 관리
- **클러스터 관리**: 카프카 클러스터내에 서버가 추가되거나 제외될때 그 정보를 클러스터 내부 서버들이 공유하는데 사용.

- **리더 선출**: 한 브로커에서 장애 발생시 대기 중인 서버 중에서 자동 선정하여 새로 선정된 서버가 해당 서비스를 이어받아 처리할 수 있게 함으로써 서비스가 중단되지 않도록 함. 
> 데이터 쓰기를 하는 서버를 리더(leader)라고 하고 리더를 복제하며 대기하는 서버를 팔로어(follwer)라 함. 리더 선출 방식은 주키퍼 공식 사이트 참조

- **락, 동기화 서비스**: 카프카 클러스터에 쓰기 연산이 빈번할 경우 데이터 불일치가 발생, 이 때 클러스터 전체를 동기화해 (락) 불일치 해소.
   

#### 카프카 브로커 개요 및 구성 옵션

카프카 브로커는 토픽들을 관리한다. 카프카 브로커의 구성 옵션은 매우 다양하며 그 중 필수 옵션은 다음과 같다.

1. broker.id
> 모든 카프카는 반드시 고유한 식별자를 가져야 한다. 기본값은 0이다. 

2. port
> 카프카 실행에 사용되는 TCP 포트이다. 기본값은 9092이다.

3. zookeeper.connect
> 브로커의 메타 데이터를 저장하기 위해 사용되는 주키퍼의 위치를 나타낸다. 기본값은 localhost:2181 이다.

4. log.dirs
> 카프카는 모든 메시지를 로그 세그먼트 파일에 모아서 디스크에 저장하는데 그 경로를 나타낸다. 여러 경로를 쉼표로 구분하여 지정할 수 있고 지정한 모든 경로에 파일을 저장한다.

5. num.recovery.threads.per.data.dir
> 카프카는 스레드 풀을 사용하여 로그 세그먼트를 처리한다. 기본적으로 로그 디렉터리당 하나의 쓰레드만 사용되고, 브로커의 시작과 종료 시에만 사용되므로 병행 처리를 위해 많은 수를 설정하는 것이 좋다.
> - 브로커가 정상적으로 실행: 각 파티션의 로그 세그먼트 파일을 열기 위해 쓰레드 사용됨.
> - 브로커 장애 발생 이후 다시 시작: 각 파티션의 로그 세그먼트 파일을 검사하고 불필요한 부분 삭제하기 위해 사용.
> - 브로커 정상적으로 종료: 파일을 정상적으로 닫기 위해 스레드 사용.

6. auto.create.topics.enable
> 다음과 같은 상황에서 브로커는 자동으로 하나의 토픽을 생성한다. 기본값은 True 이다.
> - 프로듀서가 토픽에 메시지를 쓸때
> - 컨슈머가 토픽의 메시지를 읽기 시작할때
> - 클라이언트에서 토픽 메타데이터를 요청할때 

### 4.2. 프로듀서

#### 프로듀서 개요
- 프로듀서는 토픽에 메시지를 쓰는(Write)하는 주체이다.
- 카프카를 큐, 메시지 버스, 데이터 스토리지 플랫폼 중 어떤 용도로 사용하든 카프카에 데이터를 쓰는 프로듀서와 데이터를 읽는 컨슈머 애플리케이션을 작성해서 카프카를 사용하게 된다.
- 개발자는 카프카의 프로듀서와 컨슈머를 카프카 클라이언트 API를 통해 애플리케이션을 개발한다.
- 프로듀서 API는 매우 직관적이며 문제 상황, 제약 조건(예: 최대한 빠른 처리시간, 메시지 유실 및 복제 허용)에 따라 프로듀서 API의 구성을 변경할 수 있다.

![title](./assets/high_level_producer_api.png)

- 위의 그림은 브로커에 데이터를 전송할 때 프로듀서가 내부적으로 처리하는 작업(점선으로 된 사각형)을 보여준다.
    1. 브로커에 전송하려는 메시지를 갖는 ProducerRecord 객체 생성(kafka-python 에선 객체 생성이 필요 없고, Java 사용시 필요), 이 객체에는 토픽과 데이터를 포함한다. (필요에 따라 키와 파티션을 지정할 수 있다. 이에 대한 내용은 뒷부분에 설명)
    2. 이 메시지 객체들이 네트워크로 전송될 수 있도록 바이트 배열로 직렬화(직렬처리기)한다.
    3. 바이트 배열로 직렬화된 메시지 객체는 파티셔너 컴포넌트(클래스)에 전달된다. 만일 1번 과정에서 특정 파티션을 선택할 경우 파티셔너가 특별한 처리를 하지 않고 지정된 파티션에 전달한다, 그러나 파티션이 지정되지 않았다면 이 메시지 객체의 키를 기준으로 파티셔너 컴포넌트가 하나의 파티션을 선택하고 그 파티션에 전달한다.
    4. 3번 과정까지 종료되면 해당 메시지 객체(ProducerRecord 객체)가 저장될 토픽과 파티션을 알게 되었고 이후 같은 토픽과 파티션으로 전송될 메시지들을 모은 레코드 배치(batch)에 추가하며, 별개의 스레드가 그 배치를 카프카 브로커에 전송한다.
    5. 브로커는 수신된 레코드의 메시지를 저장한 후 응답을 전송한다. 이때 메시지를 성공적으로 쓰면 RecordMetadata 객체를 반환한다. 만약 메시지 쓰기를 실패하면 에러를 반환하며, 에러를 수신한 프로듀서에서 재전송을 몇 번 더 시도할 수 있다.

#### 프로듀서의 메시지 전송 방식

프로듀서 객체는 아래의 3가지 방법을 통해 브로커에 메시지를 쓴다.  
만약 처리량(throghput)을 더 늘리고 싶다면 같은 프로듀서를 멀티 스레드로 구성한다.

1. 전송 후 망각(Fire-and-forget)
> 가장 간단한 방법이며, 프로듀스 객체의 `send()` 메서드로 메세지를 전송만 하고 성공 또는 실패 여부에 따른 후속 조취를 취하지 않는 방법이다. 카프카는 가용성이 높고 전송에 실패할 경우 프로듀서가 자동으로 재전송을 시도하므로 대부분의 경우에 성공적으로 메시지가 전송된다. 다만 이 방법을 사용하면 일부 메시지가 유실 될 수도 있다.

2. 동기식 전송(Synchronous send)
> 프로듀스 객체의 `send()` 메서드로 메시지를 전송하면 `Future` 객체가 반환된다. 이후 `Future` 객체의 `get()` 메서드를 곧바로 호출하면 컨슈머에서  메시지 쓰기 작업이 완료될 때까지 기다렸다가 브로커로부터 처리 결과가 반환되므로 send()가 성공적으로 수행되었는지 알 수 있다.

3. 비동기식 전송(Asynchronous send)
> `send()` 메서드를 호출할 때 콜백(callback) 메서드를 구현한 객체(카프카에서 제공하는 CallBack 인터페이스를 구현한 클래스 인스턴스로, 뒤에 나오는 예제 코드를 보면 구체적으로 알 수 있다.)를 매개변수로 전달한다. 이 객체에 구현된 콜백 메서드는 카프카 브로커로부터 응답을 받을 때 자동으로 호출되므로 send()가 성공적으로 수행되었는지 알 수 있다.

```Python
# 실제 실습 코드는 맨 마지막에 있습니다.
from kafka import KafkaProducer
from kafka.errors import KafkaError

# kafka-python에서 Producer 객체의 send() 함수는 기본적으로 비동기식 전송을 한다. 
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])

# 1. 전송 후 망각
future = producer.send('test-topic', b'test-data')

# 2. 동기식 전송
future = producer.send('test-topic', b'test-data')
try:
    record_metadata = future.get(timeout=10)
except KafkaError:
    print('전송 에러')

# 3. 비동기식 전송
def on_send_success(record_metadata):
    print('전송 성공: ', record_metadata)
def on_send_error(excp):
    print('전송실패: ', excp)
future = producer.send('test-topic', b'test-data')\
    .add_callback(on_send_success).add_errorback(on_send_error)


producer.flush() # 모든 비동기식 전송 메시지가 보내질때까지 블럭킹됨
```

#### 프로듀서 구성 옵션
문제 상황, 제약조건(예: 최대한 빠른 처리시간, 메시지 유실 및 복제 허용)에 따라 [프로듀스의 구성 옵션](http://kafka.apache.org/documentation.html#producerconfigs)을 변경할 수 있다.

아래 옵션들은 프로듀서 구성 옵션중 *핵심적*으로 입력해야할 옵션들이다. 

1. bootstrap.servers
> 프로듀서가 사용하는 브로커의 `host:port`(서버 이름과 포트번호) 목록을 이 속성에 설정한다.  
> 모든 브로커의 정보가 필요하지 않고 2개 정도가 권장된다. 최초로 연결된 브로커로부터 추가적인 정보를 얻기 때문이고, 2개를 하는 이유는 첫번째 브로커가 중단되는 경우를 방지하기 위해서이다.

2. key.serializer
> 프로듀서가 생성하는 레코드(ProducerRecord 객체, 파이썬에서는 객체 생성이 필요 없고 `send()`함수가 해당됨)의 메시지 키를 직렬화 하기 위해 사용되는 클래스. 

3. value.serializer
> 프로듀서가 생성하는 레코드(ProducerRecord 객체, 파이썬에서는 객체 생성이 필요 없고 `send()`함수가 해당됨)의 메시지 값을 직렬화 하기 위해 사용되는 클래스. 

```Python
# 실제 실습 코드는 맨 마지막에 있습니다.
from kafka import KafkaProducer
from kafka.errors import KafkaError

# value 직렬화로 msgpack(매우 빠르고 효율적으로 binary 직렬화 하는 모듈) 사용 
producer = KafkaProducer(
    bootstrap_servers=['127.0.0.1:9092'],
    key_serializer=msgpack.dumps,
    value_serializer=msgpack.dumps
)

# value 직렬화로 json 사용 
producer = KafkaProducer(
    bootstrap_servers=['127.0.0.1:9092'],
    key_serializer=lambda m: json.dumps(m).encode('ascii'),
    value_serializer=lambda m: json.dumps(m).encode('ascii')
)
```

### 4.3. 토픽, 파티션(Partition), 오프셋(Offset)
- 토픽(Topic): 카프카 클러스터에 데이터를 관리할 시 그 기준이 되는 개념이다. 하나의 토픽은 1개 이상의 파티션(Partition)으로 구성되어 있다.

- 파티션(Partition): 메시지의 로드벨런싱을 위해 토픽을 논리적으로 분할해 놓은 개념이다. 실제로 데이터가 round-robin으로 분산되서 쓰기/읽기 되는 곳은 이곳이다.

![partition](./assets/partition.png)

> 여기서 주의해야 할 점은 파티션이 토픽의 복제(replication)가 아니고 (논리적인) 분할이라는 점이다. 파티션의 복제는 `replica-factor` 옵션을 구성하여 할 수 있다.  
> 한 토픽의 파티션의 수가 3개이고, `replica-factor` 값이 3 라면 총 9개의 파티션이 각 카프카 브로커에 생성된다.

![producing-to-partitions](assets/producing-to-partitions.png)

- 오프셋(offset): 각 컨슈머가 파티션을 읽을때 사용하는 인덱스 이다.

> 토픽에 데이터가 쓰여질때 메시지들은 자동으로 파티션에 분산되어 ([4.2. 프로듀서 api 설명 그림 참조](#4.2.-프로듀서)) 저장되며 브로커는 이 메시지들을 디스크에 보존한다. 즉 메시지를 소비해도 메시지는 그대로 보존된다.  
> 따라서 소비자가 어디까지 메시지를 소비했는지 알 수 있는 상대적 위치값이 필요하며 이것이 오프셋이다. 오프셋은 각 파티션을 관리하는 컨슈머 마다 하나씩 가지고 있다.

![consumer_group](./assets/consumer_group.png)

### 4.4. 컨슈머, 컨슈머 그룹
#### 컨슈머 개요
- 컨슈머는 토픽의 메세지를 소비하는 주체이다. 
- 토픽의 메시지들을 브로커로부터 pull 하여 처리함으로 컨슈머의 처리 능력에 따라 스스로 조절해가면서 소비할 수 있다. 
- 소비를 했다는 표시는 해당 topic내의 각 파티션에 존재하는 offset의 위치를 통해서 이루어진다.
- 만약 컨슈머에 장애가 발생하여 죽었다가 다시 살아나도 이전에 소비했던 offset 위치를 쥬키퍼가 기억하고 관리하고 있기에 전에 마지막으로 읽었던 위치에서 부터 다시 읽어들일 수 있다. 즉, fail-over에 대한 신뢰가 존재한다.
> 만약 컨슈머에 장애 발생후 컨슈머가 읽고 있었던 파티션을 처음부터 다시 읽게 하거나, 아니면 최근 들어온 메시지 순서부터 읽게 구성할 수 있다.  

#### 컨슈머 구성 옵션
문제 상황, 제약조건(예: 최대한 빠른 처리시간, 메시지 유실 및 복제 허용)에 따라 [컨슈머의 구성 옵션](http://kafka.apache.org/documentation.html#consumerconfigs)을 변경할 수 있다.

아래 옵션들은 프로듀서 구성 옵션중 *핵심적*으로 입력해야할 옵션들이다. 

1. bootstrap.servers
> 컨슈머 사용하는 브로커의 `host:port`(서버 이름과 포트번호) 목록을 이 속성에 설정한다.  
> 모든 브로커의 정보가 필요하지 않고 2개 정도가 권장된다. 최초로 연결된 브로커로부터 추가적인 정보를 얻기 때문이고, 2개를 하는 이유는 첫번째 브로커가 중단되는 경우를 방지하기 위해서이다.

2. key.serializer
> 컨슈머가 소비하는 레코드의 키를 직렬화 하기 위해 사용되는 클래스. 

3. value.serializer
> 컨슈머가 소비하는 레코드의 값을 직렬화 하기 위해 사용되는 클래스. 

4. group.id
> 컨슈머 그룹 구별을 위한 고유한 ID.

#### 컨슈머 그룹
- 컨슈머의 집합을 구성하는 단위이다. 
- 카프카에서는 컨슈머 그룹단위로 데이터를 처리하며 컨슈머 그룹 안의 컨슈머 수만큼 파티션들의 데이터를 분산처리하게 된다.

하나의 토픽 내에 파티션4개와 컨슈머 1개가 존재할 경우  
![p4c1](./assets/p4c1.png)

하나의 토픽 내에 파티션4개와 컨슈머 2개가 존재할 경우  
![p4c2](./assets/p4c2.png)

하나의 토픽 내에 파티션4개와 컨슈머 4개가 존재할 경우  
![p4c4](./assets/p4c4.png)

하나의 토픽 내에 파티션4개와 컨슈머 5개가 존재할 경우  
![p4c5](./assets/p4c5.png)
> 이때 파티션을 할당받지 못한 컨슈머는 **대기** 상태로 존재하며 만약 컨슈머 중 하나에 장애가 발생하면 그 역활을 대신 이어받을 수 있다.

```Python
# 실제 실습 코드는 맨 마지막에 있습니다.
from kafka import KafkaConsumer
from kafka.errors import KafkaError

# 컨슈머 그룹은 'Consumer_Group1' 그룹에 참가.
# key/value 역직렬화로 msgpack사용
consumer = KafkaConsumer(
    bootstrap_servers=['127.0.0.1:9092'],
    group_id='Consumer_Group1'
    key_deserializer=msgpack.loads,
    value_deserializer=msgpack.loads
)

# TopicT1 구독
consumer.subscribe(['TopicT1'])

# 메시지를 읽어옴
for msg in consumer:
    print(msg)
```

---

# 5. Hellow Kafka!

#### 5.1. 도커 설치 확인
```Bash
$ docker -v

$ docker-compose -v
```

#### 5.2. 카프카 설치

로컬환경에서 원활한 개발을 위해 도커-컴포즈를 사용하여 쥬키퍼 클러스터(노드3개)와 카프카 클러스터(노드3개)를 구성한다.

![title](./assets/kafka-cluster-step2.gif)

컴포즈 파일 내부의 파라미터들의 의미는 다음과 같다.

#### Docker compose file
##### 주키퍼 클러스터 
```Yml
version: '2'
services:
  # 쥬키퍼 클러스터 구성 (만약 프로덕션 환경이라면 최소한 3개의 쥬키퍼 노드를 구성해야한다.)
  zookeeper-1:
    image: confluentinc/cp-zookeeper:latest
    hostname: zookeeper-1
    ports:
      - "12181:12181"
    environment:
      ZOOKEEPER_SERVER_ID: 1
      ZOOKEEPER_CLIENT_PORT: 12181
      ZOOKEEPER_TICK_TIME: 2000
      ZOOKEEPER_INIT_LIMIT: 5
      ZOOKEEPER_SYNC_LIMIT: 2
      ZOOKEEPER_SERVERS: zookeeper-1:12888:13888;zookeeper-2:22888:23888;zookeeper-3:32888:33888

  zookeeper-2:
    image: confluentinc/cp-zookeeper:latest
    hostname: zookeeper-2
    ports:
      - "22181:22181"
    environment:
      ZOOKEEPER_SERVER_ID: 2
      ZOOKEEPER_CLIENT_PORT: 22181
      ZOOKEEPER_TICK_TIME: 2000
      ZOOKEEPER_INIT_LIMIT: 5
      ZOOKEEPER_SYNC_LIMIT: 2
      ZOOKEEPER_SERVERS: zookeeper-1:12888:13888;zookeeper-2:22888:23888;zookeeper-3:32888:33888

  zookeeper-3:
    image: confluentinc/cp-zookeeper:latest
    hostname: zookeeper-3
    ports:
      - "32181:32181"
    environment:
      ZOOKEEPER_SERVER_ID: 3
      ZOOKEEPER_CLIENT_PORT: 32181
      ZOOKEEPER_TICK_TIME: 2000
      ZOOKEEPER_INIT_LIMIT: 5
      ZOOKEEPER_SYNC_LIMIT: 2
      ZOOKEEPER_SERVERS: zookeeper-1:12888:13888;zookeeper-2:22888:23888;zookeeper-3:32888:33888
```
실제 주키퍼 서버 설정 파일은 `{ZOOKEEPER_HOME}/conf/zoo.cfg` 경로에 있다.

Docker-Compose 파일 안의 파라미터들의 의미는 다음과 같다.
- 주키퍼(zookeeper-1 ~ 3)
    1. `X3888, X2888 포트` : 주키퍼 내부 노드끼리 통신을 위한 포트 (2888포트는 동기화, 3888은 리더 선출을 위한 포트).
    
    > 리더 선출: 분산처리 시스템에선 리더와 팔로워가 존재하는데, 주로 리더에서 전체 시스템의 동기화를 위한 연산(읽기/쓰기)작업을 하고 팔로워는 리더의 복제본이다.  
    > 만약 리더에서 장애 발생시 리더의 역활을 대신할 서버를 선출하는 것이 리더 선출 이다.  
    > 주키퍼에 대해 더 자세히 알아보려면 [주키퍼 소개 및 아키텍쳐](https://engkimbs.tistory.com/660) 참조.
    
    2. `X2181 포트` : 주키퍼 클러스터를 사용하는 클라이언트와 통신을 위한 포트.
    
    > 포트 앞에 X라는 접두사가 붙은 이유는 한 서버에서 3개의 쥬피터 노드가 띄워질 경우 포트 충돌을 막기 위해서이다. 각각 다른 서버에서 띄운다면 접두사를 없애도 된다.
    
    3. `ZOOKEEPER_SERVER_ID` : 주키퍼 노드을을 구분하기 위한 고유한 id.
    
    4. `ZOOKEEPER_CLIENT_PORT` : 주키퍼 클러스터에 접근할 클라이언트 포트
    
    5. `ZOOKEEPER_TICK_TIME` : 주키퍼에서 사용되는 세션 타임 아웃 시간 단위 (밀리세컨드)
    
    6. `ZOOKEEPER_SYNC_LIMIT` : 처음 주키퍼의 follower가 leader와의 연결 시도시 가지는 tick 제한 횟수. 제한 횟수가 넘으면 타임아웃
    
    > 실제 시간은 ZOOKEEPER_TICK_TIME * ZOOKEEPER_INIT_LIMIT = 2 * 5 = 10초
    
    7. `ZOOKEEPER_SYNC_LIMIT` : follower가 leader와 연결된 후, 계속 ensemble 안에서 leader와 동기화되기 위한 tick 제한 횟수. 제한 횟수가 넘으며 타임아웃
    
    > 실제 시간은 ZOOKEEPER_TICK_TIME * ZOOKEEPER_SYNC_LIMIT = 2 * 2 = 4초
    
    8. `ZOOKEEPER_SERVERS` : 주키퍼가 클러스터를 이루기 위한 모든 서버의 정보를 가진다. 
    
    > 예를 들어 `zookeeper-3:32888:33888` zookeeper-3 은 IP주소, 32888은 노드들끼리 통신 포트, 33888은 리더 선출을 위한 포트이다.
    
#### 카프카 브로커 클러스터
```Yml
  kafka-1:
    image: confluentinc/cp-kafka:latest
    hostname: kafka-1
    ports:
      - "19092:19092"
    depends_on:
      - zookeeper-1
      - zookeeper-2
      - zookeeper-3
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper-1:12181,zookeeper-2:12181,zookeeper-3:12181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka-1:9092,PLAINTEXT_HOST://localhost:19092
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
 
  kafka-2:
    image: confluentinc/cp-kafka:latest
    hostname: kafka-2
    ports:
      - "29092:29092"
    depends_on:
      - zookeeper-1
      - zookeeper-2
      - zookeeper-3
    environment:
      KAFKA_BROKER_ID: 2
      KAFKA_ZOOKEEPER_CONNECT: zookeeper-1:12181,zookeeper-2:12181,zookeeper-3:12181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka-2:9092,PLAINTEXT_HOST://localhost:29092
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
 
  kafka-3:
    image: confluentinc/cp-kafka:latest
    hostname: kafka-3
    ports:
      - "39092:39092"
    depends_on:
      - zookeeper-1
      - zookeeper-2
      - zookeeper-3
    environment:
      KAFKA_BROKER_ID: 3
      KAFKA_ZOOKEEPER_CONNECT: zookeeper-1:12181,zookeeper-2:12181,zookeeper-3:12181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka-3:9092,PLAINTEXT_HOST://localhost:39092
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
```

Docker-Compose 파일 안의 파라미터들의 의미는 다음과 같다.
- 카프카 브로커
    
    1. `X9092 포트`: 카프카가 컨슈머, 프로듀서 API와 통신하기 위한 포트 
    
    2. `KAFKA_BROKER_ID`: 카프카 브로커를 구분하는 고유한 id 이다.
    
    3. `KAFKA_ZOOKEEPER_CONNECT`: 주키퍼 서버 정보이다. 기본적으로 2개 이상의 주키퍼 서버 정보를 지정한다. 
    
    > 하나의 주키퍼 서버에 접속하면 주키퍼가 가지고있는 클러스터 메타데이터(다른 카프카 서버들)을 모두 가져올 수 있다.  
    그리고 최소 두개의 주키퍼 서버를 입력하는 이유는 하나의 서버에서 장애가 있을 경우 두번째 서버에 접속하여 메타데이터를 가져오기 위함이다.
    
    4. `KAFKA_ADVERTISED_LISTENERS`: Kafka 브로커를 가리키는 사용 가능 주소 목록.
    
    5. `KAFKA_LISTENER_SECURITY_PROTOCOL_MAP`: 리스너 이름당 사용할 보안 프로토콜
    
    6. `KAFKA_INTER_BROKER_LISTENER_NAME`: 내부 내트워크에서 사용할 리스너목록
    
    > 카프카 클러스터 내에서 브로커들끼리 내부 네트워크로 통신을 하게 되는데(주키퍼에 모든 브로커들의 메타 데이터가 저장되어있고 개별 브로커들은 그 데이터를 Watch 하고 있음) 내부 통신시 리스닝할 포트를 정의하는 부분이다.
    > ![docker01](assets/docker01.png)
    > 사진에 표시된 부분과 우리의 도커 컴포즈 파일이 살짝 다른데 우리의 설정에서는 내부 네트워크 통신시 9092 포트를 사용하고, 외부에서 접속시 카프카 브로커별 19092, 29092, 39092 포트를 사용한다. 



#### 5.3. Docker compose UP!

```Bash
$ cd kafka_cluster_docker_compose
$ docker-compose up
```

### Example:

![kafka-configuration-example-3](./assets/kafka_configuration_example_3.jpg)

카프카의 토픽을 생성하는 방법은 크게 auto 와 manual 이 있다. 
1. auto
    1. 프로듀서가 만들어지지 않은 토픽에 데이터를 전송할때
    2. 컨슈머가 만들어지지 않은 토픽에 데이터를 전송할때
2. manual
    1. 카프카 관리 sh 스크립트를 사용하여 직접 주키퍼에게 토픽 생성 명령을 전송할때
    2. (docker 사용시) docker compose 파일에서 브로커 컨테이너 생성시 토픽을 자동으로 생성하는 변수(`KAFKA_CREATE_TOPIC:`)를 설정할 수 있음.

> 권장사항은 오토 토픽 생성 `auto.create.topics.enable` 기능을 끄고 카프카 관리 스크립트로 직접 주키퍼에게 명령을 전송하는 방법이다.

이 예제에서는 2.1. 방식을 사용해서 토픽을 생성하겠다.

#### Prerequisites

1. kafka-python 설치
```Bash
conda install -c conda-forge kafka-python 
```


2. msgpack 설치
```Bash
conda install -c anaconda msgpack-python
```


3. 카프카 클러스터, 쥬키퍼 클러스터 구성
```Bash
$ cd kafka-docker-compose
$ docker-compose up
```


4. 카프카 관리 스트립트 설치
```Bash
$ cd ~
$ wget http://apache.mirror.cdnetworks.com/kafka/2.2.1/kafka_2.11-2.2.1.tgz
$ tar -xvzf kafka_2.11-2.2.1.tgz
$ cd kafka_2.11-2.1.tgz
```


5. `advice_topic` 토픽 생성 및 파티션 2개, 리플리케이션 팩터 2개로 생성.
```Bash
$ sh bin/kafka-topics.sh --create --zookeeper localhost:12181 --replication-factor 2 --partitions 2 --topic advice_topic
Created topic advice_topic2.
```


6. `advice_topic` 생성 확인
```Bash
$ sh bin/kafka-topics.sh --describe --zookeeper localhost:12181 --topic
Topic:advice_topic2     PartitionCount:2        ReplicationFactor:2     Configs:
        Topic: advice_topic2    Partition: 0    Leader: 1       Replicas: 1,2   Isr: 1,2
        Topic: advice_topic2    Partition: 1    Leader: 2       Replicas: 2,3   Isr: 2,3
```

#### 5.4. Producer Sample

In [8]:
import json
import random

from kafka import KafkaProducer
from kafka.errors import KafkaError

producer = KafkaProducer(
    bootstrap_servers=[
        '127.0.0.1:19092' # 
    ],
    key_serializer=lambda m: json.dumps(m).encode('ascii'),
    value_serializer=lambda m: json.dumps(m).encode('ascii')
)

print('파티션 정보:', producer.partitions_for('advice_topic'))

파티션 정보: {0, 1}


In [9]:
for i in range(10):
    producer.send('advice_topic', i, partition=random.choice([0,1]))

#### 5.5. Consumer Sample

In [10]:
import json
import multiprocessing

from kafka import KafkaConsumer
from kafka.errors import KafkaError

def consumer_group(consumer_name='', consumer_group_id='', consumer_q=None):
    try:
        print(consumer_name + ' is spawned')
        consumer = KafkaConsumer(
            auto_offset_reset='earliest',
            bootstrap_servers=[
                '127.0.0.1:19092'
            ],
            group_id=consumer_group_id,
            key_deserializer=lambda m: json.loads(m.decode('ascii')),
            value_deserializer=lambda m: json.loads(m.decode('ascii'))
        )

        consumer.subscribe(['advice_topic'])

        for message in consumer:
            print ("[%s] - %s:%d:%d: key=%s value=%s" % (consumer_name ,message.topic, message.partition, message.offset, message.key, message.value))
            consumer_q.put(message)
            
    except KafkaError as err:
        raise err
        return False

with multiprocessing.Pool(processes=3) as pool:
    print("multiprocessing - spawn 3 consumer")
    m = multiprocessing.Manager()
    consumer1_q = m.Queue()
    consumer2_q = m.Queue()
    consumer3_q = m.Queue()
    
    consumer1 = pool.apply_async(
        consumer_group, 
        ('consumer1', 'consumer_group1', consumer1_q)
    )
    consumer2 = pool.apply_async(
        consumer_group, 
        ('consumer2', 'consumer_group1', consumer2_q)
    )
    consumer3 = pool.apply_async(
        consumer_group, 
        ('consumer3', 'consumer_group2', consumer3_q)
    )
    
    try:
        consumer1.get(timeout=20)
    except TimeoutError:
        print("consumer1 - Consuming done")
        
    try:
        consumer2.get(timeout=20)
    except TimeoutError:
        print("consumer2 - Consuming done")
    
    try:
        consumer3.get(timeout=20)
    except TimeoutError:
        print("consumer3 - Consuming done")

multiprocessing - spawn 3 consumer
consumer1 is spawned
consumer2 is spawned
consumer3 is spawned
[consumer1] - advice_topic:0:6: key=None value=0
[consumer1] - advice_topic:0:7: key=None value=1
[consumer1] - advice_topic:0:8: key=None value=2
[consumer1] - advice_topic:0:9: key=None value=8
[consumer1] - advice_topic:0:10: key=None value=9
[consumer3] - advice_topic:0:6: key=None value=0
[consumer3] - advice_topic:0:7: key=None value=1
[consumer3] - advice_topic:0:8: key=None value=2
[consumer3] - advice_topic:0:9: key=None value=8
[consumer3] - advice_topic:0:10: key=None value=9


TimeoutError: 

In [5]:
# 컨슈머 그룹 2
while consumer3_q.empty() != True:
    message = consumer3_q.get()
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))

advice_topic:0:0: key=None value=3
advice_topic:0:1: key=None value=4
advice_topic:0:2: key=None value=5
advice_topic:0:3: key=None value=7
advice_topic:0:4: key=None value=8
advice_topic:0:5: key=None value=9
advice_topic:1:0: key=None value=0
advice_topic:1:1: key=None value=1
advice_topic:1:2: key=None value=2
advice_topic:1:3: key=None value=6


In [6]:
# 컨슈머 그룹 1
while consumer2_q.empty() != True:
    message = consumer2_q.get()
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))

advice_topic:0:0: key=None value=3
advice_topic:0:1: key=None value=4
advice_topic:0:2: key=None value=5
advice_topic:0:3: key=None value=7
advice_topic:0:4: key=None value=8
advice_topic:0:5: key=None value=9


In [7]:
# 컨슈머 그룹 1
while consumer1_q.empty() != True:
    message = consumer1_q.get()
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))

advice_topic:1:0: key=None value=0
advice_topic:1:1: key=None value=1
advice_topic:1:2: key=None value=2
advice_topic:1:3: key=None value=6


#### 5.6. Stress testing
##### 1. 브로커중 하나에 장애 발생시
```Bash
$ docker stop kafka-docker-compose_kafka-2_1
```

다시 5.4, 5.5 Example 실행해서 결과를 비교.

##### 2. 주키퍼중 하나에 장애 발생시
```Bash
$ docker stop kafka-docker-compose_zookeeper-1_1
```

다시 5.4, 5.5 Example 실행해서 결과를 비교.

---

## Appendix. 카프카 관리 스크립트
[이곳](https://kafka.apache.org/downloads)에서 카프카를 관리 스크립트를 다운로드받거나 아래 명령어를 참조한다.

> 도커 컴포즈를 사용할 경우 도커에서 사용되는 카프카 버전에 맞춰 카프카를 다운로드 받아야 한다.  

1. 카프카 관리 스크립트 다운로드

```Bash
$ cd ~
$ wget http://apache.mirror.cdnetworks.com/kafka/2.2.1/kafka_2.11-2.2.1.tgz
$ tar -xvzf kafka_2.11-2.2.1.tgz
$ cd kafka_2.11-2.1.tgz
```

2. 기본적인 명령어 모음, 파라미터를 요구사항에 맞게 변경하여 사용  

```Bash
# 토픽 생성
$ sh bin/kafka-topics.sh --create --zookeeper localhost:12181 --replication-factor 1 --partitions 2 --topic my_topic_test

# 토픽 삭제
$ sh bin/kafka-topics.sh --delete --zookeeper localhost:12181 --topic my_topic_test

# 테스트 컨슈머 생성
$ sh bin/kafka-console-consumer.sh --bootstrap-server localhost:19092 --topic my_topic-test --from-beginning

# 테스트 프로듀서 생성
$ sh bin/kafka-console-producer.sh --broker-list localhost:19092 --topic my_topic_test

# 토픽 파티션 변경
$ sh bin/kafka-topics.sh --alter --zookeeper localhost:12181 --topic my_topic_test --partitions 2

# 토픽 정보 확인
$ sh bin/kafka-topics.sh --describe --zookeeper localhost:12181 --topic my_topic_test
```

---

## Appendix. 단일 카프카 설치

1. kafka 다운로드
```Bash
$ cd ~
$ wget http://apache.tt.co.kr/kafka/2.2.0/kafka_2.12-2.2.0.tgz
$ tar -xzf kafka_2.12-2.2.0.tgz
$ cd kafka_2.12-2.2.0
```


2. 주키퍼 서버 실행
```Bash
$ sh bin/zookeeper-server-start.sh config/zookeeper.properties
```


3. 카프카 서버 실행
```Bash
$ sh bin/kafka-server-start.sh config/server.properties
```


4. 토픽생성
```Bash
$ sh bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test
```


5. 프로듀서 - 메시지 생성
```Bash
$ sh bin/kafka-console-producer.sh --broker-list localhost:9092 --topic test
This is a message
This is another message
```


6. 컨슈머 - 메시지 소비
```Bash
$ sh  bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic test --from-beginning
This is a message
This is another message
```

## Reference
##### 카프카 관련

[카프카 완벽 가이드(en).pdf](https://www.confluent.io/wp-content/uploads/confluent-kafka-definitive-guide-complete.pdf)

https://thepracticaldeveloper.com/2018/11/24/spring-boot-kafka-config/

https://engkimbs.tistory.com/691

https://medium.com/zaneiru-tech-life-blog/apache-kafka-%EC%B9%B4%ED%94%84%EC%B9%B4-%EC%9D%98-%ED%8A%B9%EC%A7%95-%EB%B0%8F-%EB%AA%A8%EB%8D%B8-b3d3f7e6a4df

https://www.popit.kr/kafka-consumer-group/

https://engkimbs.tistory.com/560

https://engkimbs.tistory.com/691

https://medium.com/zaneiru-tech-life-blog/apache-kafka-%EC%B9%B4%ED%94%84%EC%B9%B4-%EC%9D%98-%ED%8A%B9%EC%A7%95-%EB%B0%8F-%EB%AA%A8%EB%8D%B8-b3d3f7e6a4df

https://zzsza.github.io/data/2018/06/15/apache-kafka-intro/
##### 주키퍼 관련
https://d2.naver.com/helloworld/583580